## Overview of the Project:
##### POTATO is a prototype that tracks tweets from U.S. voters between 2016 and 2023,
##### allowing users to search for specific terms like "COVID" and get aggregated information about the people 
#### who tweeted those terms. The system uses technologies like Docker, Elasticsearch, Streamlit, and Python, 
#### and the major challenges include data ingestion and quick query results. There’s also a need to ensure user privacy.




In [1]:

import pandas as pd 
import numpy as np

In [2]:
tsv_file = 'correct_twitter_201904.tsv'
data = pd.read_csv(tsv_file, sep='\t')

In [3]:
df=data


In [4]:
df.isnull().sum()

id                                   0
event                                0
ts1                                  0
 ts2                                 0
from_stream                          0
directly_from_stream                 0
from_search                          0
directly_from_search                 0
from_quote_search                    0
directly_from_quote_search           0
from_convo_search                    0
directly_from_convo_search           0
from_timeline_search                 0
directly_from_timeline_search        0
text                                 0
lang                                 0
author_id                            0
author_handle                        0
created_at                           0
conversation_id                      0
possibly_sensitive                   0
reply_settings                       0
source                               0
author_follower_count                0
retweet_count                        0
reply_count              

In [5]:
df.describe()

,id,author_id,conversation_id,author_follower_count,retweet_count,reply_count,like_count,quote_count,replied_to,replied_to_author_id,replied_to_follower_count,quoted,quoted_author_id,quoted_follower_count,retweeted,retweeted_author_id,retweeted_follower_count,mentioned_author_ids,mentioned_handles
count,8.803700e+04,8.803700e+04,8.803700e+04,8.803700e+04,88037.000000,88037.000000,8.803700e+04,88037.000000,6.675000e+03,3.850000e+03,3.850000e+03,2.052000e+03,1.776000e+03,1.776000e+03,5.152900e+04,5.152900e+04,5.152900e+04,0.0,0.0
mean,1.128375e+18,2.840685e+17,1.128223e+18,5.871965e+04,613.836353,8.710633,1.643042e+02,9.802674,1.127070e+18,2.636418e+17,1.018013e+06,1.124363e+18,1.489432e+17,2.149497e+06,1.123201e+18,2.561212e+17,1.589441e+06,NaN,NaN
std,3.001479e+15,4.376297e+17,7.439132e+15,1.200196e+06,4047.938227,382.716992,8.811544e+03,470.460294,2.286399e+16,4.301651e+17,6.457768e+06,4.342901e+16,3.496499e+17,8.885742e+06,6.133647e+16,4.351598e+17,5.231865e+06,NaN,NaN
min,1.101535e+18,1.357300e+04,1.597973e+17,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,1.597973e+17,6.616130e+05,0.000000e+00,3.576774e+16,7.592510e+05,9.000000e+00,1.572632e+09,8.943000e+03,0.000000e+00,NaN,NaN
25%,1.126285e+18,2.148085e+08,1.126257e+18,2.140000e+02,0.000000,0.000000,0.000000e+00,0.000000,1.125241e+18,7.296255e+07,5.170000e+02,1.126942e+18,2.725816e+07,7.012000e+03,1.126080e+18,2.265021e+07,5.042000e+03,NaN,NaN
50%,1.128714e+18,1.635683e+09,1.128709e+18,5.950000e+02,20.000000,0.000000,0.000000e+00,0.000000,1.128014e+18,6.361114e+08,2.780000e+03,1.127323e+18,3.724831e+08,1.060500e+04,1.128478e+18,4.786582e+08,2.276000e+04,NaN,NaN
75%,1.130296e+18,7.860916e+17,1.130281e+18,1.761000e+03,286.000000,0.000000,0.000000e+00,0.000000,1.130909e+18,7.616023e+17,2.627000e+04,1.128954e+18,9.328426e+08,5.033700e+05,1.129886e+18,7.339592e+17,8.797590e+05,NaN,NaN
max,1.134308e+18,1.134151e+18,1.134308e+18,1.143075e+08,415500.000000,81701.000000,1.368193e+06,76727.000000,1.134301e+18,1.132513e+18,1.143075e+08,1.134308e+18,1.130235e+18,1.143074e+08,1.134304e+18,1.133221e+18,7.141746e+07,NaN,NaN


In [6]:
### Drop columns with too many missing values (optional)

df_clean = df.drop(columns=['replied_to', 'replied_to_author_id', 'quoted', 'quoted_author_id',
                            'retweeted', 'retweeted_author_id', 'mentioned_author_ids', 'mentioned_handles'])


In [7]:
# Fill missing values for numerical columns (if appropriate)
df_clean['hashtags'] = df_clean['hashtags'].fillna('')
df_clean['urls'] = df_clean['urls'].fillna('')
df_clean['media_keys'] = df_clean['media_keys'].fillna('')


In [8]:
# Drop rows where essential fields are missing
df_clean = df_clean.dropna(subset=['text', 'author_id', 'created_at'])

In [9]:
# Convert the 'created_at' column to datetime with UTC handling
df_clean['created_at'] = pd.to_datetime(df_clean['created_at'], utc=True)

In [10]:
# Step 1: Convert 'created_at' to datetime with timezone handling (UTC)
df_clean['created_at'] = pd.to_datetime(df_clean['created_at'], utc=True)

# Step 2: Extract useful components (date, hour) from 'created_at'
df_clean['date'] = df_clean['created_at'].dt.date
df_clean['hour'] = df_clean['created_at'].dt.hour

## Lowercase all text for consistent querying.
## Remove URLs and other unnecessary characters.

In [11]:
## import the regular expression (regex) module, which provides support for working with regular expressions.
import re 

# Function to clean tweet text
def clean_text(text):
    text = text.lower()  # Lowercase all text
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    return text

# Apply cleaning to the 'text' column
df_clean['cleaned_text'] = df_clean['text'].apply(clean_text)

In [12]:
# Remove duplicate tweets
df_clean = df_clean.drop_duplicates(subset=['author_id', 'text', 'created_at'])

In [13]:
##Number of tweets per day

tweets_per_day = df_clean.groupby('date')['text'].count()
print(tweets_per_day)

date
2019-03-01       1
2019-03-02       2
2019-03-03       3
2019-03-04       1
2019-03-05       3
              ... 
2019-05-27    1025
2019-05-28    1695
2019-05-29    2026
2019-05-30    2077
2019-05-31     417
Name: text, Length: 83, dtype: int64


In [14]:
##count the number of unique author_id.
unique_users_per_day = df_clean.groupby('date')['author_id'].nunique()
print(unique_users_per_day)

date
2019-03-01       1
2019-03-02       2
2019-03-03       3
2019-03-04       1
2019-03-05       3
              ... 
2019-05-27     915
2019-05-28    1445
2019-05-29    1740
2019-05-30    1651
2019-05-31     376
Name: author_id, Length: 83, dtype: int64


In [15]:
##Filter by tweets containing a specific term and calculate the average like_count.  

search_term = 'music'
filtered_data = df_clean[df_clean['cleaned_text'].str.contains(search_term)]
average_likes = filtered_data['like_count'].mean()
print(f'Average likes for tweets containing "{search_term}": {average_likes}')

Average likes for tweets containing "music": 161.40853048983672


In [16]:
## User with the most tweets containing a specific term:
##Find the user who tweeted the most about a given term.

In [17]:
top_user = filtered_data.groupby('author_id')['text'].count().idxmax()
print(f'User with the most tweets containing "{search_term}": {top_user}')

User with the most tweets containing "music": 118301422


In [18]:
## Group by hour to see the distribution of tweets posted at different times of the day
tweets_by_hour = df_clean.groupby('hour')['text'].count()
print(tweets_by_hour)


hour
0     4243
1     3961
2     3826
3     3653
4     3102
5     2708
6     2246
7     2162
8     2049
9     1980
10    2030
11    2251
12    2777
13    3490
14    4102
15    4297
16    4616
17    5000
18    5203
19    4877
20    5574
21    5213
22    4468
23    4209
Name: text, dtype: int64


In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def home():
    return "Hello, Flask is running in Jupyter!"

if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
